## 循环神经网络


### 1. 序列模型的应用

    语音识别：将输入的语音信号直接输出相应的语音文本信息。无论是语音信号还是文本信息均是序列数据。
    音乐生成：生成音乐乐谱。只有输出的音乐乐谱是序列数据，输入可以是空或者一个整数。
    情感分类：将输入的评论句子转换为相应的等级或评分。输入是一个序列，输出则是一个单独的类别。
    DNA序列分析：找到输入的DNA序列的蛋白质表达的子序列。
    机器翻译：两种不同语言之间的想换转换。输入和输出均为序列数据。
    视频行为识别：识别输入的视频帧序列中的人物行为。
    命名实体识别：从输入的句子中识别实体的名字。

### 2. 数学符号

    输入x：如“Harry Potter and Herminone Granger invented a new spell.”(以序列作为一个输入)，x<t>表示输入x中的第t个符号。
    输出y：如“1 1 0 1 1 0 0 0 0”（人名定位），同样，用y<t>表示输出y中的第t个符号。
    Tx用来表示输入x的长度；
    Ty用来表示输出y的长度；
    x(i)<t>表示第i个输入样本的第t个符号，其余同理。
    利用单词字典编码来表示每一个输入的符号：如one-hot编码等，实现输入x和输出y之间的映射关系。

### 3. 循环神经网络模型
传统标准的神经网络：
对于学习X和Y的映射，我们可以将输入的序列X以某种方式进行字典编码以后，如one-hot编码，输入到一个多层的深度神经网络中，最后得到对应的输出Y

但是，结果表明这种方法并不好，主要是存在下面两个问题：

    输入和输出数据在不同的例子中可以有不同的长度；
    这种朴素的神经网络结果并不能共享从文本不同位置所学习到的特征。（如卷积神经网络中学到的特征的快速地推广到图片其他位置）
    
#### 循环神经网络：
循环神经网络作为一种新型的网络结构，在处理序列数据问题上则不存在上面的两个缺点。在每一个时间步中，循环神经网络会传递一个激活值到下一个时间步中，用于下一时间步的计算。如下图所示： 
![title](img/xhwl.png)

这里需要注意在零时刻，我们需要编造一个激活值，通常输入一个零向量，有的研究人员会使用随机的方法对该初始激活向量进行初始化。同时，上图中右边的循环神经网络的绘制结构与左边是等价的。

循环神经网络是从左到右扫描数据的，同时共享每个时间步的参数。

    Wax管理从输入x<t>到隐藏层的连接，每个时间步都使用相同的Wax，同下；
    Waa管理激活值a<t>到隐藏层的连接；
    Wya管理隐藏层到激活值y<t>的连接。
上述循环神经网络结构的缺点：每个预测输出y<t>仅使用了前面的输入信息，而没有使用后面的信息。Bidirectional RNN（双向循环神经网络）可以解决这种存在的缺点。
    

### 4 前向传播和反向传播
#### 前向传播
![title](img/fxhwl.png)

#### 反向传播
我们需要定义一个损失函数，如下：
![title](img/bxhwl.png)
上式表示将每个输出的损失进行求和即为整体的损失函数。反向传播算法按照前向传播相反的方向进行导数计算，来对参数进行更新。其中比较特别的是在RNN中，从右向左的反向传播计算是通过时间来进行，如穿越时间的反向计算。

### 5. 不同类型的RNN
对于RNN，不同的问题需要不同的输入输出结构。
many-to-many（Tx=Ty）：
这种情况下的输入和输出的长度相同，是上面例子的结构，如下图所示：

![title](img/m2mRnn.png)

many-to-one：
如在情感分类问题中，我们要对某个序列进行正负判别或者打星操作。在这种情况下，就是输入是一个序列，但输出只有一个值：
![title](img/m2oRnn.png)

one-to-many：
如在音乐生成的例子中，输入一个音乐的类型或者空值，直接生成一段音乐序列或者音符序列。在这种情况下，就是输入是一个值，但输出是一个序列：
![title](img/o2mRnn.png)

many-to-many（Tx≠Ty）：
我们上面介绍的一种RNN的结构是输入和输出序列的长度是相同的，但是像机器翻译这种类似的应用来说，输入和输出都是序列，但长度却不相同，这是另外一种多对多的结构：
![title](img/m2m2Rnn.png)

 ### 6.RNN的梯度消失
RNN在NLP中具有很大的应用价值，但是其存在一个很大的缺陷，那就是梯度消失的问题。例如下面的例句中：

    The cat, which already ate ………..，was full；
    The cats, which already ate ………..，were full.
在这两个句子中，cat对应着was，cats对应着were，（中间存在很多很长省略的单词），句子中存在长期依赖（long-term dependencies），前面的单词对后面的单词有很重要的影响。但是我们目前所见到的基本的RNN模型，是不擅长捕获这种长期依赖关系的。
对于梯度消失问题，在RNN的结构中是我们首要关心的问题，也更难解决；虽然梯度爆炸在RNN中也会出现，但对于梯度爆炸问题，因为参数会指数级的梯度，会让我们的网络参数变得很大，得到很多的Nan或者数值溢出，所以梯度爆炸是很容易发现的，我们的解决方法就是用梯度修剪，也就是观察梯度向量，如果其大于某个阈值，则对其进行缩放，保证它不会太大。

### 7. 各种RNN模型
####  GRU单元
门控循环单元（Gated Recurrent Unit, GRU）改变了RNN的隐藏层，使其能够更好地捕捉深层次连接，并改善了梯度消失的问题。

简化的GRU 单元：
我们以时间步从左到右进行计算的时候，在GRU单元中，存在一个新的变量称为c，（代表cell）,作为“记忆细胞”，其提供了长期的记忆能力。

![title](img/GRU1.png)

完整的GRU 单元：
完整的GRU单元还存在另外一个门，以决定每个时间步的候选值，公式如下：
![title](img/GRU2.png)

#### LSTM
GRU能够让我们在序列中学习到更深的联系，长短期记忆（long short-term memory, LSTM）对捕捉序列中更深层次的联系要比GRU更加有效。
LSTM中，使用了单独的更新门Γu和遗忘门Γf，以及一个输出门Γo，其主要的公式如下：
![title](img/LSTM1.png)

LSTM单元的可视化图如下所示：
![title](img/LSTM2.png)

其中，在实际使用时，几个门值不仅仅取决于a<t−1>和x<t>，还可能会取决于上一个记忆细胞的值c<t−1>，这也叫做偷窥孔连接。GRU和LSTM的选择可以根据具体的情况来。
    
#### 双向RNN
双向RNN（bidirectional RNNs）模型能够让我们在序列的某处，不仅可以获取之间的信息，还可以获取未来的信息。


#### 深层RNNs
与深层的基本神经网络结构相似，深层RNNs模型具有多层的循环结构，但不同的是，在传统的神经网络中，我们可能会拥有很多层，几十层上百层，但是对与RNN来说，三层的网络结构就已经很多了，因为RNN存在时间的维度，所以其结构已经足够的庞大。
![title](img/deeprnn.png)

### 8.实例
LSTM 生成爵士音乐
https://github.com/jisungk/deepjazz

## 自然语言处理与词嵌入
### 1.词嵌入
用不同的特征来对各个词汇进行表征，相对与不同的特征，不同的单词均有不同的值
![title](img/wordembeddings.png)

### 2. 词嵌入的迁移学习：
有了词嵌入，就可以让我们能够使用迁移学习，通过网上大量的无标签的文本中学习到的知识，应用到我们少量文本训练集的任务中。下面是做词嵌入迁移学习的步骤：
第一步：从大量的文本集合中学习word Embeddings（1-100B words），或者从网上下载预训练好的词嵌入模型；
第二步：将词嵌入模型迁移到我们小训练集的新任务上；
第三步：可选，使用我们新的标记数据对词嵌入模型继续进行微调。

### 3. 词嵌入的特性
类比推理特性：
词嵌入还有一个重要的特性，它还能够帮助实现类比推理。如下面的例子中，通过不同词向量之间的相减计算，可以发现不同词之间的类比关系，man——woman、king——queen
计算词与词之间的相似度，实际上是在多维空间中，寻找词向量之间各个维度的距离相似度。
eman-ewoman 与 eking-e?相似度最大得到e？

### 4. 嵌入矩阵
在我们要对一个词汇表学习词嵌入模型时，实质上就是要学习这个词汇表对应的一个嵌入矩阵E。当我们学习好了这样一个嵌入矩阵后，通过嵌入矩阵与对应词的one-hot向量相乘，则可得到该词汇的embedding

### 5. Word2Vec
Word2Vec算法是一种简单的计算更加高效的方式来实现对词嵌入的学习。
#### Skip-grams：
在Skip-grams模型中，我们需要抽取上下文（Content）和目标词（Target）配对，来构造一个监督学习问题。
上下文不一定是要目标词前面或者后面离得最近的几个单词，而是随机选择一个词作为上下文，同时在上下文的一定距离范围内随机选择另外一个词作为目标词。构造这样一个监督学习问题的目的，并不是想要解决监督学习问题本身，而是想要使用这个问题来学习一个好的词嵌入模型。
模型流程：
![title](img/word2vec.png)

#### 存在的问题：
计算速度的问题，如在上面的Softmax单元中，我们需要对所有10000个整个词汇表的词做求和计算，计算量庞大。
简化方案：使用分级softmax分类器（相当于一个树型分类器，每个节点都是可能是一个二分类器），其计算复杂度是前面的log|v|级别。在构造分级softmax分类器时，一般常用的词会放在树的顶部位置，而不常用的词则会放在树的更深处，其并不是一个平衡的二叉树。
#### 如何采样上下文：
在构建上下文目标词对时，如何选择上下文与模型有不同的影响。

对语料库均匀且随机地采样：使得如the、of、a等这样的一些词会出现的相当频繁，导致上下文和目标词对经常出现这类词汇，但我们想要的目标词却很少出现
。
采用不同的启发来平衡常见和不常见的词进行采样。这种方法是实际使用的方法。
### 6. 负采样
Skip-grams模型可以帮助我们构造一个监督学习任务，将上下文映射到目标词上，从而让我们能够学习到一个实用的词嵌入模型。但是其缺点就是softmax计算的时间复杂度较高。下面介绍一种改善的学习问题：负采样。其能够做到的和Skip-grams模型相似，但其学习算法更加有效。

新的学习问题：

    定义一个新的学习问题：预测两个词之间是否是上下文-目标词对，如果是词对，则学习的目标为1；否则为0。
    使用k次相同的上下文，随机选择不同的目标词，并对相应的词对进行正负样本的标记，生成训练集。
    建议：小数据集，k=5~20；大数据集，k=2~5。
    最后学习x——y的映射关系。

### 7. GloVe 词向量
GloVe（global vectors for word representation）词向量模型是另外一种计算词嵌入的方法，虽然相比下没有Skip-grams模型用的多，但是相比这种模型却更加简单。

### 8. 情感分类
情感分类就是通过一段文本来判断这个文本中的内容是否喜欢其所讨论的内容，这是NLP中最重要的模块之一。
情感分类任务存在的一个问题就是只有很小的数据集，缺乏训练样本。但是在使用了词嵌入后，则能够带来很好的效果，足以训练一个良好的情感分类模型。
#### 平均值或和的模型：

    获取一个训练好的词嵌入矩阵E；
    得到每个词的词嵌入向量，并对所有的词向量做平均或者求和；
    输入到softmax分类器中，得到最后的输出y^；
    缺点：没有考虑词序，可能会导致多数的积极词汇削弱前面消极词汇的影响，从而造成错误的预测。
#### RNN模型：

    获取一个训练好的词嵌入矩阵E；
    得到每个词的词嵌入向量，输入到many-to-one的RNN模型中；
    通过最后的softmax分类器，得到最后的输出y^。
    优点：考虑了词序，效果好很多。

### 9. 词嵌入消除偏见
当下机器学习或者人工智能算法已经被应用到做一些非常重要的决策中，因此我们需要尽可能地保证其不受非预期形式的偏见的影响，如性别、种族歧视等等。下面介绍一些在词嵌入中消除偏见的办法。

目前的偏见问题：
以一些预料库中学习到的词嵌入向量，会发现学习到的词向量存在下面一些具有性别、种族等偏见，这反映了人们在历史的写作中存在的这种社会偏见：

#### 消除偏见的方法：
    定义偏见的方向：如性别 
    对大量性别相对的词汇进行相减并求平均
    通过平均后的向量，则可以得到一个或多个偏见趋势相关的维度，以及大量不相关的维度；
    中和化：对每一个定义不明确的词汇，进行偏见的处理，如像doctor、babysitter这类词；通过减小这些词汇在得到的偏见趋势维度上值的大小；
    均衡：将如gradmother和gradfather这种对称词对调整至babysitter这类词汇平衡的位置上，使babysitter这类词汇处于一个中立的位置，进而消除偏见。

## 序列模型和注意力机制 
### 1. 基础模型
sequence to sequence 模型：
sequence to sequence 模型最为常见的就是机器翻译，假如这里我们要将法语翻译成英文：

    输入：x<1>，x<2>，⋯，x<Tx>；这里每个x<t>均为对应法语句子中的每个单词；
    输出：y<1>，y<2>，⋯，y<Ty>；这里每个y<t>均为对应英语句子中的每个单词；
网络结构：many-to-many RNN网络结构。
对于机器翻译的序列对序列模型，如果我们拥有大量的句子语料，则可以得到一个很有效的机器翻译模型。模型的前部分使用一个编码网络来对输入的法语句子进行编码，后半部分则使用一个解码网络来生成对应的英文翻译。

image to sequence 模型：
与上面的这种编解码类似的还有就是图像描述的应用。

    输入：图像；
    输出：描述图像的句子；
    
网络结构：CNN结构学习图像的编码，RNN结构解码输出对应的句子。

### 2. 挑选最可能的句子
通过输入的法语句子，模型将会告诉我们各种英文句子输出所对应的可能性
对于各种可能的翻译结果，我们并不是要从得到的分布中进行随机取样，而是要找到一个使得条件概率最大化的英文句子作为输出。所以在设计机器翻译模型的时候，一个重要的步骤就是设计一个合适的算法，找到使得条件概率最大化的的结果。目前最通用的算法就是：束搜索（Beam Search）。
### 3. 集束搜索（Beam search）
Beam search 算法：
这里我们还是以法语翻译成英语的机器翻译为例：
Step 1：对于我们的词汇表，我们将法语句子输入到编码网络中得到句子的编码，通过一个softmax层计算各个单词（词汇表中的所有单词）输出的概率值，通过设置集束宽度（beam width）的大小如3，我们则取前3个最大输出概率的单词，并保存起来。

Step 2：在第一步中得到的集束宽度的单词数，我们分别对第一步得到的每一个单词计算其与单词表中的所有单词组成词对的概率。并与第一步的概率相乘，得到第一和第二两个词对的概率。有3×10000个选择，（这里假设词汇表有10000个单词），最后再通过beam width大小选择前3个概率最大的输出对；

Step 3~Step T：与Step2的过程是相似的，直到遇到句尾符号结束。

### 4. 集束搜索的改进
长度归一化：
对于集束搜索算法，我们的目标就是最大化下面的概率：
![title](img/bs1.png)
上面的得到的每一项一般都是很小的概率值，大量很小的概率值进行相乘，最后会得到更小的值，可能会造成数值下溢。所以在实践中，我们不会最大化上面这个公式的乘积，而是取log值，变成log求和最大值，得到一个数值上更加稳定的算法，公式如下：
![title](img/bs2.png)
另外，我们还可以通过对上面的目标进行归一化，使其得到更好的效果。相比直接除以输出单词长度的值，可以使用更加柔和的方式：在Ty上加上一个指数α，如α=0.7，通过调整其大小获得更加好的效果。
![title](img/bs3.png)
通过上面的目标，选取得分最大的句子，即为我们的模型最后得到的输出结果。
集束搜索讨论：
Beam width：B的选择，B越大考虑的情况越多，但是所需要进行的计算量也就相应的越大。在常见的产品系统中，一般设置B = 10，而更大的值（如100，1000，…）则需要对应用的领域和场景进行选择。

相比于算法范畴中的搜索算法像BFS或者DFS这些精确的搜索算法，Beam Search 算法运行的速度很快，但是不能保证找到目标准确的最大值。
### 5. 集束搜索的误差分
集束搜索算法是一种近似搜索算法，也被称为启发式搜索算法。它的输出不能保证总是可能性最大的句子，因为其每一步中仅记录着Beam width为3或者10或者100种的可能的句子。
所以，如果我们的集束搜素算法出现错误了要怎么办呢？我们如何确定是算法出现了错误还是模型出现了错误呢？此时集束搜索算法的误差分析就显示出了作用。

#### 例子：
同样以法语句子的英文翻译为例子，我们人类对法语句子的翻译如中间的句子，而我们的模型输出的翻译如下面的句子。通过我们的模型，我们分别计算人类翻译的概率P(y∗|x）以及模型翻译的概率P(y^|x)，比较两个概率的大小，通过比较我们就能知道是因为Beam Search 算法的问题还是RNN模型的问题。

    P(y∗|x)>P(y^|x)的情况：Beam search算法选择了y^，但是y∗却得到了更高的概率，所以Beam search 算法出错了；
    P(y∗|x)<=P(y^|x)的情况：翻译结果y∗相比y^要更好，但是RNN模型却预测P(y∗|x)<P(y^|x)，所以这里是RNN模型出现了错误。

### 6. Bleu Score
对于机器翻译系统来说，一种语言对于另外一种语言的翻译常常有多种正确且合适的翻译，我们无法做到像图像识别一样有固定准确度答案，所以针对不同的翻译结果，往往很难评估那一个结果是更好的，哪一个翻译系统是更加有效的。这里引入Bleu score 用来评估翻译系统的准确性。（Bleu, bilingual evaluation understudy）

#### 评估机器翻译：
如下面的法语翻译的例子，我们有两种不同的翻译，但是两种翻译都是正确且较好的翻译结果。Bleu score 的评估理念是观察机器生成的翻译结果中的每一个词是否出现在至少一个人工翻译结果的参考之中。（这些参考位于开发集或者测试集中）。

    精确度：观察输出结果的每一个词是否出现在参考中。但是对于图中机器翻译的糟糕的结果，精确度确非常高。
    改良的精确度：将每个单词设置一个得分上限（单个参考句子中出现的最大的次数，如图中的the单词的上限为2）。
    
#### 二元词组的Bleu score：
与单个词的评估相似，这里我们以两个相邻的单词作为一个二元词组来进行Bleu得分评估，得到机器翻译的二元词组的得分和其相应的得分上限，进而得到改进的精确度。

### 7. 注意力模型
长句子存在的问题：

利用我们前面的编码和解码的RNN模型，我们能够实现较为准确度机器翻译结果。对于短句子来说，其性能是十分良好的，但是如果是很长的句子，翻译的结果就会变差。
对于我们人类进行人工翻译的时候，我们所做的也不是像编码解码RNN模型一样记忆整个输入句子，再进行相应的输出，因为记忆整个长句子是很难的，所以我们是一部分一部分地进行翻译。编码解码RNN模型的结构，其Bleu score会在句子长度超过一定值的时候就会下降

注意力模型的简介：

对于法语翻译英语的例子中，针对每个单词的输出，一般来说与某个输出相关的或者有较大影响的单词应该是集中在附近的几个单词或者说是某些部分。所以注意力模型会在输入的每个输入的信息块上计算注意力权重，不同的权重对每一步的输出结果有不同的注意力影响。

### 8. 语音识别
语音识别是将一段音频信号转化为相应的文本信息。

#### 语音识别问题：
在语音识别问题上，借鉴于人耳的处理过程，会将音频信号转化为该音频信号的声谱图（时间和频率的图，不同颜色代表不同的频谱能量的大小），再将声谱图作为特征送到相应的算法中进行处理。

之前的语音识别的系统中，语言学家构造了音位来学习语音识别系统。但是随着深度学习的发展，在end-to-end的模型中，这种音位的表示法已经不重要了。我们完全可以做到直接从语音信号到文本信息的转化，无需人工特征的设计。

#### 注意力模型的语音识别：
为了构建一个语音识别的系统，我们可以应用前面介绍的注意力模型

#### CTC 损失函数的语音识别：
同时，另外一种效果较好的就是使用CTC损失函数的语音识别模型。（CTC，Connectionist temporal classification）。
对于语音识别系统来说，我们的输入是音频信号，而输出是文本信息，对于音频信号，我们以小的时间间隔进行频率采样，可能对于一个10s的语音片段，我们就能够得到1000个特征的输入片段，而往往我们的输出仅仅是几个单词。
对于上面的问题，在CTC损失函数中，允许我们的RNN模型输出有重复的字符和插入空白符的方式，强制使得我们的输出和输入的大小保持一致

### 9. 触发字检测
随着目前深度学习的发展，越来越多的智能设备需要通过一些关键语音来进行唤醒的操作，相应的这些系统可以被称作触发字检测系统。
触发字检测算法：
一种可以简单应用的触发字检测算法，就是使用RNN模型，将音频信号进行声谱图转化得到图像特征或者使用音频特征，输入到RNN中作为我们的输入。而输出的标签，我们可以以触发字前的输出都标记为0，触发字后的输出则标记为1。